# 웹 데이터 스크래핑

In [ ]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc

# 그림 사이즈 설정
plt.rc('figure', figsize=(12,8))
font_fname = 'c:/windows/fonts/gulim.ttc' # 윈도우용
#font_fname = '/System/Library/Fonts/Supplemental/AppleGothic.ttf' # 맥용
font_name = font_manager.FontProperties(fname=font_fname).get_name()
rc('font', family=font_name)

## HTML Data

#### Simple example

In [ ]:
!more "data\example.html"

In [ ]:
from lxml.html import parse
from urllib.request import urlopen

In [ ]:
# 데이터 읽어 오기
                                     
    

In [ ]:
# <a href='http://www.postech.ac.kr'>POSTECH</a><br>
# <a href='http://ime.postech.ac.kr'>Department of Industrial and Management Engineering</a><br>
# <a href='http://aim.postech.ac.kr'>Analytics & Information Management Lab</a><br>

# <a>로 시작하는 부분 가져오기
links = doc.findall('.//a')
print("links: " + str(links))

lnk = links[0]  # 첫 link 가지고 오기
print("\n href: "+lnk.get('href')) # 연결 링크 가지고 오기

lnk.text_content() # 텍스트 가지고 오기
print("\n content: "+lnk.text_content())

# <a>에 있는 모든 링크 가지고 오기
urls = [lnk.get('href') for lnk in doc.findall('.//a')]
print("\n urls: "+ str(urls))

# <a>에 있는 모든 텍스트 가지고 오기
lnks = [lnk.text_content() for lnk in doc.findall('.//a')]
print("\n links: "+ str(lnks))

In [ ]:
# 테이블 처리 예제
# 모든 <table> 가지고 오기



In [ ]:
table = tables[0]             # 테이블리스트에서 첫번째 테이블 선택
rows = table.findall('.//tr') # <table>에서 <tr>(행) 추출
rows

In [ ]:
############################################
# <tr>에 있는 <td>의 값 추출 함수 정의
def _unpack(row, kind='td'):
    elts = row.findall('.//%s' % kind)
    return [val.text_content() for val in elts]

In [ ]:
# _unpack() 예제: 세번째 줄의 데이터값 가지고 오기
_unpack(rows[2], kind = 'td')

In [ ]:
############################################
# 테이블을 dataframe으로 변환하는 함수
from pandas.io.parsers import TextParser

def parse_table_data(table):
    rows = table.findall('.//tr')
    header = _unpack(rows[0], kind = 'th')
    data = [_unpack(r) for r in rows [1:]]
    return TextParser(data, names=header).get_chunk()

In [ ]:
# parse_table_data() 예제 : # 테이블을 dataframe으로 변환



## 네이버 주식 데이터 읽어 오기

In [ ]:
from pandas.io.parsers import TextParser
from lxml.html import parse
from urllib.request import urlopen

code='005930' # 삼성전자
#code='055550' # 신한지주

parsed = parse(urlopen('http://finance.naver.com/item/sise_day.nhn?code='+code)) # 웹 문서 접근 및 구조 추출
doc = parsed.getroot()              # 가지고 온 웹 문서의 가장 상위 위치 추출 
tables = doc.findall('.//table')    # 테이블을 리스트 추출
table = tables[0]                   # 테이블을 리스트에서 첫 테이블 추출
data = parse_table_data(table)      # 테이블을 dataframe으로 변환

data

# [Lab] "신한지주" 주식 1에서 50페이지까지 데이터로 분석 수행

In [ ]:
# 여러 페이지에서 주식 데이터 추출하여 DataFrame에 추가
# http://finance.naver.com/item/sise_day.nhn?code=005930&page=2

code='005930' # 삼성전자
#code='055550' # 신한지주

start = 1 # 시작 페이지
end = 40  # 종료 페이지

stock_data = pd.DataFrame() # 주식 데이터를 담을 DataFrame 준비

for page in range(start, end):
    url = 'http://finance.naver.com/item/sise_day.nhn?code='+code+'&page='+ str(page) # 페이지 고려한 접속 주소 생성
    parsed = parse(urlopen(url))               # 웹 문서 접근 및 구조 추출
    docsamsung = parsed.getroot()              # 가지고 온 웹 문서의 가장 상위 위치 추출 
    tables = docsamsung.findall('.//table')    # 테이블을 리스트 추출
    table = tables[0]                          # 테이블을 리스트에서 첫 테이블 추출
    data = parse_table_data(table)             # 테이블을 dataframe으로 변환
    
    stock_data = pd.concat([stock_data, data]) # 추출된 DataFrame을 결합
    
stock_data     # 추출된 DataFrame

## 날짜별 주가 변동 그래프

In [ ]:
# 날짜별 주가 변동 그래프를 그리기 위해 '날짜'를 인덱스로 지정



In [ ]:
                            # 데이터 정보 보기

In [ ]:
stock_data2['시가'].astype(float) # 데이터를 숫자로 변환 시도 --> 에러 발생?

In [ ]:
# ,를 제거하고 값을 숫자로 변환하는 방법
# dataframe comma python data type conversion
stock_data2['시가'].str.replace(',', '').astype(float)

In [ ]:
# 데이터에서 ,를 제거하고 값을 숫자로 변환
for column in stock_data2.columns:
    stock_data2[column] = stock_data2[column].str.replace(',', '').astype(float)

stock_data2.head()

In [ ]:
# '날짜'로 데이터 정렬



In [ ]:
# 주가 그래프 생성


In [ ]:
# '거래량'에 대한 스케일 조정
stock_data2['거래량2']=stock_data2['거래량']        # '거래량' 컬럼을 보전하기 위해 '거래량2' 생성
stock_data2['거래량']=stock_data2['거래량2']/1500   # '거래량' 스케일 조정
                                                   # '거래량2'는 제외하고 '거래량' 컬럼까지 그래프 생성

In [ ]:
# 양쪽의 y 축을 사용하여 '종가'와 '거래량' 데이터 표시
stock_data2.종가.plot(style='r', label='종가')
stock_data2.거래량.plot(secondary_y=True, style='g', label='거래량2')

## 주가 이동 평균 그래프 생성 및 등락폭과 거래량 관계 분석 

In [ ]:
# 주가 이동 평균 그래프
                                                        # '종가' 컬럼 추출

stock_data3['ema10'] = stock_data3['종가'].ewm(10).mean()   # 10일 이동 평균선 계산
stock_data3['ema30'] = stock_data3['종가'].ewm(30).mean()   # 30일 이동 평균선 계산
stock_data3['ema60'] = stock_data3['종가'].ewm(60).mean()   # 60일 이동 평균선 계산

                                           # 그래프 생성

In [ ]:
# 등락폭과 거래량 관계 분석
stock_data4 = stock_data2.loc[:,['전일비','거래량','종가']]   # '전일비', '거래량', '종가' 컬럼 추출
stock_data4.plot.scatter(x='전일비', y='거래량', c='종가', marker='s', cmap='viridis')